![MSE Logo](https://moodle.msengineering.ch/pluginfile.php/1/core_admin/logo/0x150/1643104191/logo-mse.png)

# AnTeDe Lab 4: Search Engine with the Vector Space Model

## Summary
The aim of this lab is to build a simple document search engine based on TF-IDF document vectors. 

The lab is inspired by a notebook designed by [Kavita Ganesan](https://github.com/kavgan/nlp-in-practice/blob/master/TF-IDF/Keyword%20Extraction%20with%20TF-IDF%20and%20SKlearn.ipynb).

<font color='green'>Please answer the questions in green within this notebook, and submit the completed notebook under the corresponding homework on Moodle.</font>

In [1]:
import os
import nltk  # on Colab, you mind find it helpful to run nltk.download('popular') to install packages

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

import gensim
import numpy as np
import pandas as pd
from nltk.corpus import stopwords, wordnet

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import linear_kernel
from gensim import models, corpora, similarities
import contractions

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/davebrunner/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/davebrunner/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/davebrunner/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/davebrunner/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/davebrunner/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
# Import TextProcessor.py from local directory structure
import sys

module_path = os.path.abspath(os.path.join('../week 1'))
if module_path not in sys.path:
    sys.path.append(module_path)

from TextPreprocessor import *

The data used in this lab is a set of 300 documents selected from the Australian Broadcasting Corporation's news mail service. It consists of texts of headline stories from around the years 2000-2001.  This is a shortened version of the Lee Background Corpus [described here](http://www.socsci.uci.edu/~mdlee/lee_pincombe_welsh_document.PDF).  It is available as test data in the **gensim** package, so you do not need to download it separately.

The following code will load the documents into a Pandas dataframe.

In [3]:
# Code inspired from:
# https://github.com/bhargavvader/personal/blob/master/notebooks/text_analysis_tutorial/topic_modelling.ipynb

test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
text = open(lee_train_file).read().splitlines()
data_df = pd.DataFrame({'text': text})

The following code will run our in-house Text Preprocessor provided in the `TextPreprocessor.py` file, and documented in the `MSE_AnTeDe_TextPreprocessingDemo.ipynb` notebook provided in Lab 1 (see Lab 1 archive on Moodle for both files).

<font color='green'> **Question**: Please enhance the code by adding special characters such as e.g., " ' ' " as stopwords and uses adjective and noun POS tag sets in the TextPreprocessor function.</font>

In [4]:
language = 'english'
stop_words = set(stopwords.words(language))
# Extend the list here:
stop_words.update(
    [',', '.', '!', '?', ';', ':', '(', ')', '[', ']', '{', '}', '``', "''", '""', '``', "''", '""', '’', '“', '”', '‘',
     '—', '–', '…', '•', '·', '°', '€', '£', '¥', '¢', '§', '©', '®', '™', '¶', '†', '‡', '‰', '№', 'Ω', '℮', '→', '↔'])
processor = TextPreprocessor(  # these are only a few of the options of TextPreprocessor (see code for more)
    language=language,
    pos_tags={wordnet.ADJ, wordnet.NOUN},
    stopwords=stop_words,
)

In [5]:
data_df['processed'] = processor.transform(data_df['text'])

/Users/davebrunner/.local/share/virtualenvs/TSM_AnTeDe-RXa4k3O7/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


We can now look at a few examples of processed texts.

In [6]:
print(data_df['text'].iloc[136])
# see the single 'a' is missing in the processed and removed by the TextPreprocessor
print(data_df['processed'].iloc[136])

A new report suggests the costs of an aging Australian population have been exaggerated. The report issued by the Australia Institute says a detailed examination of population and health data shows an aging population will not create an unsustainable burden on a shrinking workforce. Far from being an economic and social burden, it found the majority of older people enjoyed healthy and independent lives, many making financial contributions to their families and participating in voluntary community activities. The paper challenges the assumption an older population will see health costs rise to unsustainable levels. It says rising health costs are caused mainly by factors other than aging such as the growth of medical technology, rising consumer demand and escalating prices. 
new report suggests cost australian population exaggerated report australia institute detailed examination population health data show population create unsustainable burden workforce economic social burden found ma

In [7]:
data_df.head()

,text,processed
0,Hundreds of people have been forced to vacate ...,hundred people vacate home southern highland n...
1,Indian security forces have shot dead eight su...,indian security force shot dead eight militant...
2,The national road toll for the Christmas-New Y...,national road toll christmas-new year holiday ...
3,Argentina's political and economic crisis has ...,argentina 's political economic crisis resigna...
4,Six midwives have been suspended at Wollongong...,six midwife wollongong hospital south sydney i...


## Generation of document vectors with [Scikit-learn](https://scikit-learn.org/stable)

We will use the [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) class from scikit-learn to create a vocabulary and generate word counts or *Term Frequencies* (TF).
    
The result is a  matrix representation of the counts: each column represents a _word_ in the vocabulary and each row represents a document in our dataset: the cell values are the word counts of the word in the document. 

The matrix is very sparse, because all words not appearing in a document have 0 counts.

Recommended reading for usage and differences of scikit-learn’s Tfidftransformer and Tfidfvectorizer: 

https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/

In order to start using TfidfTransformer you will first have to create a CountVectorizer to count the number of words (term frequency), limit your vocabulary size, apply stop words and etc.

In [8]:
cv = CountVectorizer(max_features=3000)  # keep only the 3000 most frequent words in the corpus
word_count_vector = cv.fit_transform(data_df['processed'])

Let's look at some words from our vocabulary:

In [9]:
feature_names = cv.get_feature_names_out()

In [10]:
print(f"length of features {len(feature_names)}")  # has the max_features value been reached?
print(feature_names[2500:2505])  # try various slices
print(np.where(feature_names == 'hundred')[0])  # find a word's index
print(feature_names[1315])  # find a word corresponding to an index
# print(cv.vocabulary_.items())

# print(cv.vocabulary_.keys())
# print(cv.vocabulary_.values())

length of features 3000
['sri' 'st' 'stabbed' 'stabilise' 'stability']
[1315]
hundred


Now, let’s check the shape of the term-document matrix, which should contain 300 documents from the Lee Corpus and 3000 terms. 

In [11]:
word_count_vector.shape

(300, 3000)

In [12]:
# Output word counts for a particular word  
count_list = np.asarray(word_count_vector.sum(axis=0))[0]
word_count_dict = dict(zip(feature_names, count_list))

res = {k: v for k, v in word_count_dict.items() if k.startswith('hundred')}
# res = {k: v for k, v in word_count_dict.items() if k.startswith('people')}
res

{'hundred': 18}

**TfidfTransformer to Compute Inverse Document Frequency (IDF)**

We now use the (sparse) matrix generated by `CountVectorizer` to compute the IDF values of each word.  Note that the IDF should in reality be based on a large and representative corpus.

In [13]:
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

The IDF values are stored in the `idf_` field of the `TfidfTransformer`.  It has the same size as the array of feature names (words).

In [14]:
print(len(tfidf_transformer.idf_))  # check length

3000


To get a glimpse of how the IDF values look, we are going to print it by placing the IDF values in a python DataFrame. The values will be sorted in ascending order.

In [15]:
# print a single idf value 
print(tfidf_transformer.idf_[np.where(cv.get_feature_names_out() == 'hundred')])  # check IDF value of a word

# print idf values in a data frame 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names_out(), columns=["idf_weights"])
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

[3.81673851]


,idf_weights
mr,1.911320
year,2.015762
australian,2.072381
new,2.101940
one,2.122143
...,...
buchanan,6.013963
steep,6.013963
steinlager,6.013963
sri,6.013963


**We define here two helper functions:**
 * the first one is a sorting function for the columns of a sparse matrix in COOrdinate format (a.k.a "ijv" or "triplet" format [explained here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html));
 * the second one extracts the feature names (*words*) and their TF-IDF values from the sorted list.

In [16]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)


def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and TF-IDF score of top n items from sorted list"""

    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]

        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    results = {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]] = score_vals[idx]

    return results

We now select a document for which we will generate TF-IDF values.  <font color="green">Please select a random document of your choice between 0 and 300.</font>

In [17]:
doc_orig = data_df['text'].iloc[111]
doc_processed = data_df['processed'].iloc[111]
print(doc_orig)
print(doc_processed)

Joseph Gutnick, the saviour and former president of the Melbourne Football Club, has failed in his bid to be re-elected as president, after stepping down earlier this year. He was also dumped from the board. Gabriel Szondy and his Team Vision ticket comprehensively beat Mr Gutnick's Melbourne First ticket, in an election which saw 75 per cent of the club's more than 16,000 members vote. Mr Szondy says he is pleased it was such a decisive victory. "We're very happy that the members have voted so overwhelmingly in support of the ticket and it hasn't been cherry-picked," he said. He attributed the victory to both the presence of former Demon's great Robert Flower on his ticket and Mr Gutnick's ill-timed attempt to settle the presidency issue mid-season. Arriving at the club's annual general meeting last night, Mr Gutnick said regardless of the outcome, he expected the result to unite the club. "It shouldn't divide the club, we should drop all our differences and work together." 
joseph gu

The next instruction generates the vector of TF-IDF values for the document using the `tfidf_transformer`.

In [18]:
tf_idf_vector = tfidf_transformer.transform(cv.transform([doc_processed]))

Next, we sort the words in the `tf_idf_vector` by decreasing TF-IDF values, first transforming the vector into a coordinate format ('coo'), and then applying our sorting function from above.  We then extract the words with the top 10 scores (and the scores) for the selected document using our second helper function from above and display them.

In [19]:
sorted_items = sort_coo(tf_idf_vector.tocoo())

topn_words = extract_topn_from_vector(feature_names, sorted_items, 10)

print(doc_orig, '\n', topn_words)

Joseph Gutnick, the saviour and former president of the Melbourne Football Club, has failed in his bid to be re-elected as president, after stepping down earlier this year. He was also dumped from the board. Gabriel Szondy and his Team Vision ticket comprehensively beat Mr Gutnick's Melbourne First ticket, in an election which saw 75 per cent of the club's more than 16,000 members vote. Mr Szondy says he is pleased it was such a decisive victory. "We're very happy that the members have voted so overwhelmingly in support of the ticket and it hasn't been cherry-picked," he said. He attributed the victory to both the presence of former Demon's great Robert Flower on his ticket and Mr Gutnick's ill-timed attempt to settle the presidency issue mid-season. Arriving at the club's annual general meeting last night, Mr Gutnick said regardless of the outcome, he expected the result to unite the club. "It shouldn't divide the club, we should drop all our differences and work together."  
 {'club'

Alternatively the TF-IDF values of the first document can be inspected by placing the TF-IDF scores from the first document into a pandas data frame.

In [20]:
feature_names = cv.get_feature_names_out()
#get tfidf vector for first document 
first_document_vector = tf_idf_vector[0]
#densify and print the scores 
df_firstdoc = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"])
print("the size of the data frame is: ", df_firstdoc.shape)

the size of the data frame is:  (3000, 1)


In [21]:
df_firstdoc.sort_values(by=["tfidf"], ascending=False).head(20)

,tfidf
club,0.465806
ticket,0.439626
gutnick,0.439626
szondy,0.219813
victory,0.151398
mr,0.139719
melbourne,0.124684
former,0.123354
saviour,0.109907
president,0.109415


In [22]:
df_firstdoc.sort_values(by=["tfidf"], ascending=False).tail(20)

,tfidf
farmer,0.0
fast,0.0
fatah,0.0
fatality,0.0
fate,0.0
father,0.0
faulkner,0.0
favour,0.0
favourable,0.0
favourite,0.0


Notice that only certain words have scores. This is because our first document does not contain all of the top 3000 tokens which then show up as zeroes. Notice that the word “a” is missing from this list. This is possibly due to internal pre-processing of CountVectorizer where it removes single characters.

The more common the word across documents, the lower its score and the more unique a word is to our first document the higher the score. So it’s working as expected except for the mysterious a that was chopped off.

<font color="green"> **Question**: Please comment briefly on the relevance of these words with respect to the document content.</font>

##### Answer
The words with the highest TF-IDF scores are the most relevant to the document content. They are the words that are most unique to the document and least common in the corpus. In this case the word "club" has a score of 0.466
The word club appears 5 times in the document and is not a really common word among general literature.
The word farmer TF-IDF of 0, because it appears in the document 0 times.

## Document-document similarity using scikit-learn

In this section, you will write the commands to compute a document-document similarity matrix over the above documents, in scikit-learn.

Please use a processing [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) and a [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) and compute the *cosine similarities* between all documents.  

Note: With Tfidftransformer you will systematically compute word counts using CountVectorizer and then compute the Inverse Document Frequency (IDF) values and only then compute the TF-IDF scores.

With Tfidfvectorizer on the contrary, you will do all three steps at once. Under the hood, it computes the word counts, IDF values, and TF-IDF scores all using the same dataset.

General guideline re. how to use Tfidftransformer over Tfidfvectorizer :

- If you need the term frequency (term count) vectors for different tasks, use Tfidftransformer.
- If you need to compute TF-IDF scores on documents within your “training” dataset, use Tfidfvectorizer
- If you need to compute TF-IDF scores on documents outside your “training” dataset, use either one, both will work.

<font color="green">**Question**: At the end, you will be asked to display the five most similar documents to the one you selected above, and compare the 1st and the 5th best results.</font>

You can use inspiration from: 
 * the above code
 * https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.XkK2ceFCe-Y
 * https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html
 * https://stackoverflow.com/questions/12118720/python-TF-IDF-cosine-to-find-document-similarity
 * https://markhneedham.com/blog/2016/07/27/scitkit-learn-tfidf-and-cosine-similarity-for-computer-science-papers

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
tfidf = TfidfVectorizer(use_idf=True, max_features=3000)
pipe = Pipeline(steps=[('pre', processor), ('tfidf', tfidf)])  # the 'processor' was defined above

<font color='green'>**Question**: Please write a function called `find_similar` which receives a `tfidf_matrix` with all similarity scores between documents, and the `index` of a document in the collection, and returns the `top_n` most similar documents to it using cosine similarity.</font>

In [25]:
def find_similar(tfidf_matrix, index, top_n=5):
    cosine_similarities = linear_kernel(tfidf_matrix[index:index + 1], tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

<font color="green">**Question**: Using the data from the Pandas form created above, please use "fit" and "transform" to generate the matrix of all document similarites called "tfidf_matrix". -- How long do these two operations take on your computer?  -- Please explain briefly in your own words what is the difference between "fit" and "transform".</font>

In [26]:
import time

start = time.time()

tfidf_matrix = pipe.fit_transform(data_df['processed'])

end = time.time()
print(f"Time to fit and transform: {end - start} seconds")

/Users/davebrunner/.local/share/virtualenvs/TSM_AnTeDe-RXa4k3O7/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Time to fit and transform: 2.2061450481414795 seconds


##### Answer
 The difference is that fit is used to learn the vocabulary from the input data and transform is used to encode the input data into a document-term matrix.

<font color="green">**Question**: Using `find_similar` and the `tfidf_matrix` please display the five most similar documents to the one you selected above, with their scores, comment them, and compare the 1st and the 5th best results.</font>

In [27]:
most_similar = find_similar(tfidf_matrix, 111)

for (index, cosine_similarity) in most_similar:
    print("-------------------------------------")
    print(f"Document {index} with similarity score {cosine_similarity}")
    print(data_df['text'].iloc[index])

-------------------------------------
Document 246 with similarity score 0.1280477164417746
The AFL's all-time leading goalkicker, Tony Lockett, will decide within the next week if he will make a comeback. Lockett has told the Sydney Swans he is interested in coming out of retirement and placing himself in this month's pre-season draft. Lockett retired at the end of the 1999 season and will turn 36 in March. Swans chief executive Kelvin Templeton says the club would welcome Lockett back. "We're not putting any undue pressure on him," Mr Templeton said. "The approach really came from Tony to us, rather than the other way." Mr Templeton says if Lockett does make a comeback, the club would not expect him to play every game. "He certainly could play a role, albeit a reduced role from the one the fans knew him to hold a couple of years back," he said. 
-------------------------------------
Document 130 with similarity score 0.08050558261090161
The condition of former Indonesian dictator Suh

<font color='green'>**Question**: Could you also use the dot product instead of the cosine similarity in the `find_similar` function?  Please answer in the following box.</font>

# Answer
I would not recommend to do that, since the dimension of the matrix is huge 300 x 4892 therefore its a very sparce matrix and the dot product would not be a good measure of similarity.

In [28]:
tfidf_matrix.shape

(300, 3000)

## Building a search engine using Gensim

<font color='green'>**Question**: Using the [tutorial on Topics and Transformations from Gensim](https://radimrehurek.com/gensim/auto_examples/core/run_topics_and_transformations.html#sphx-glr-auto-examples-core-run-topics-and-transformations-py), please implement a method that returns the documents most similar to a given query.
    
Use [Gensim's TF-IDF Model](https://radimrehurek.com/gensim/models/tfidfmodel.html) to build the model and the [MatrixSimilarity function](https://radimrehurek.com/gensim/similarities/docsim.html#gensim.similarities.docsim.MatrixSimilarity) to measure cosine similarity between documents.</font>

<font color='green'>Please write a query of your own (5-10 words), retrieve the 5 most similar documents, and comment the result.</font>

In [29]:
text = data_df['text']
cv = CountVectorizer(max_features=3000)  # keep only the 3000 most frequent words in the corpus


def get_most_similar_document(query: str):
    local_text = pd.concat([text, pd.Series([query])])
    local_processed = processor.transform(local_text)

    cv = CountVectorizer(max_features=3000)  # keep only the 3000 most frequent words in the corpus
    word_count_vector = cv.fit_transform(local_processed)
    tfidf_matrix = tfidf_transformer.transform(cv.transform(local_processed))
    most_similar = find_similar(tfidf_matrix, len(local_text) - 1)
    [print("----------\n", text.iloc[index]) for (index, _) in most_similar]
    return most_similar

get_most_similar_document("john howard")

----------
 The Prime Minister, John Howard, has revealed he will go to Indonesia for a summit meeting with Indonesian President Megawati Sukarnoputri. There have been talks underway since Mr Howard was re-elected on the timing and venue for the summit. Mr Howard has now revealed he expects to travel to Indonesia for the top level meeting in February or March. It will be his second visit to Jakarta within a year. The two leaders met in Jakarta in August shortly after President Megawati took on the role. Australia and Indonesia are co-hosting an international summit on people smuggling issues in February and those issues are expected to again be a key part of the bilateral talks. Australia and Indonesia are also discussing the resumption of military ties. President Megawati signalled the relationship between the two nations had strengthened by sending a congratulatory letter to Mr Howard after the election. 
----------
 The Prime Minister has thrown his full support behind the Governor-

[(269, 0.3460057344551972),
 (73, 0.2086701715594594),
 (203, 0.1903326282562786),
 (206, 0.11152047286012765),
 (92, 0.10785900090766765)]

## End of Lab 4
Please make sure all cells have been executed, save this completed notebook, compress it to a *zip* file, and upload it to [Moodle](https://moodle.msengineering.ch/course/view.php?id=1869).